In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import KNNImputer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from numpy import log1p
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PowerTransformer

In [2]:
data = pd.read_csv('train.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
data = data.drop(['PassengerId', "Name","Ticket","Cabin"], axis=1)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [5]:
data.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [6]:
data.nunique()

Survived      2
Pclass        3
Sex           2
Age          88
SibSp         7
Parch         7
Fare        248
Embarked      3
dtype: int64

In [7]:
data["Sex"] = data.Sex.map({"male": 1, "female": 0})
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,S
887,1,1,0,19.0,0,0,30.0000,S
888,0,3,0,NaN,1,2,23.4500,S
889,1,1,1,26.0,0,0,30.0000,C


In [ ]:

# Reshape Age column to 2D
age_values = data[['Age']].values  # Ensures (n_samples, 1)

# Scale (improves imputation stability)
scaler = StandardScaler()
scaled_age = scaler.fit_transform(age_values)

# Iterative Imputation
imputer = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=0)
imputed_scaled_age = imputer.fit_transform(scaled_age)

# Inverse transform to return to original scale
imputed_age = scaler.inverse_transform(imputed_scaled_age)

# Replace missing Age values in original DataFrame
data['Age'] = imputed_age[:, 0]
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.000000,1,0,7.2500,S
1,1,1,0,38.000000,1,0,71.2833,C
2,1,3,0,26.000000,0,0,7.9250,S
3,1,1,0,35.000000,1,0,53.1000,S
4,0,3,1,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0,13.0000,S
887,1,1,0,19.000000,0,0,30.0000,S
888,0,3,0,29.699118,1,2,23.4500,S
889,1,1,1,26.000000,0,0,30.0000,C


In [9]:
def manual_winsorize(series, lower_pct=1, upper_pct=99):
    lower = np.percentile(series.dropna(), lower_pct)
    upper = np.percentile(series.dropna(), upper_pct)
    return series.clip(lower=lower, upper=upper)

def reduce_kurtosis_without_scipy(df, kurtosis_threshold=3, lower_pct=1, upper_pct=99):
    df_transformed = df.copy()
    kurtosis_before = {}
    kurtosis_after = {}

    numeric_cols = df.select_dtypes(include=[np.number]).columns
    pt = PowerTransformer(method='yeo-johnson')

    for col in numeric_cols:
        k_before = df[col].kurtosis()
        kurtosis_before[col] = k_before

        if abs(k_before) > kurtosis_threshold:
            print(f"Transforming column '{col}' with kurtosis {k_before:.2f}")

            col_no_nan = df[col].fillna(df[col].median())
            winsorized = manual_winsorize(col_no_nan, lower_pct, upper_pct)
            reshaped = np.array(winsorized).reshape(-1,1)
            transformed = pt.fit_transform(reshaped)
            df_transformed[col] = transformed.flatten()
        else:
            df_transformed[col] = df[col]

        kurtosis_after[col] = df_transformed[col].kurtosis()

    summary = pd.DataFrame({
        'Kurtosis Before': kurtosis_before,
        'Kurtosis After': kurtosis_after
    }).sort_values(by='Kurtosis Before', ascending=False)

    return df_transformed, summary

# Usage
data, kurtosis_summary = reduce_kurtosis_without_scipy(data)
print(kurtosis_summary)
print(data)


Transforming column 'SibSp' with kurtosis 17.88
Transforming column 'Parch' with kurtosis 9.78
Transforming column 'Fare' with kurtosis 33.40
          Kurtosis Before  Kurtosis After
Fare            33.398141        1.359577
SibSp           17.880420       -1.317548
Parch            9.778125       -0.486868
Age              0.966279        0.966279
Pclass          -1.280015       -1.280015
Sex             -1.620580       -1.620580
Survived        -1.775005       -1.775005
     Survived  Pclass  Sex        Age     SibSp     Parch      Fare Embarked
0           0       3    1  22.000000  1.377032 -0.560265 -0.882726        S
1           1       1    0  38.000000  1.377032 -0.560265  1.350808        C
2           1       3    0  26.000000 -0.680021 -0.560265 -0.794217        S
3           1       1    0  35.000000  1.377032 -0.560265  1.076440        S
4           0       3    1  35.000000 -0.680021 -0.560265 -0.778627        S
..        ...     ...  ...        ...       ...       ...   

In [10]:
encoded = pd.get_dummies(data['Embarked'], prefix='Embarked').astype(int)
data = pd.concat([data.drop('Embarked', axis=1), encoded], axis=1)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.000000,1.377032,-0.560265,-0.882726,0,0,1
1,1,1,0,38.000000,1.377032,-0.560265,1.350808,1,0,0
2,1,3,0,26.000000,-0.680021,-0.560265,-0.794217,0,0,1
3,1,1,0,35.000000,1.377032,-0.560265,1.076440,0,0,1
4,0,3,1,35.000000,-0.680021,-0.560265,-0.778627,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.000000,-0.680021,-0.560265,-0.299028,0,0,1
887,1,1,0,19.000000,-0.680021,-0.560265,0.529354,0,0,1
888,0,3,0,29.699118,1.377032,1.845561,0.287979,0,0,1
889,1,1,1,26.000000,-0.680021,-0.560265,0.529354,1,0,0


In [11]:
encoded = pd.get_dummies(data['Pclass'], prefix='Pclass').astype(int)
data = pd.concat([data.drop('Pclass', axis=1), encoded], axis=1)
data

,Survived,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,0,1,22.000000,1.377032,-0.560265,-0.882726,0,0,1,0,0,1
1,1,0,38.000000,1.377032,-0.560265,1.350808,1,0,0,1,0,0
2,1,0,26.000000,-0.680021,-0.560265,-0.794217,0,0,1,0,0,1
3,1,0,35.000000,1.377032,-0.560265,1.076440,0,0,1,1,0,0
4,0,1,35.000000,-0.680021,-0.560265,-0.778627,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,1,27.000000,-0.680021,-0.560265,-0.299028,0,0,1,0,1,0
887,1,0,19.000000,-0.680021,-0.560265,0.529354,0,0,1,1,0,0
888,0,0,29.699118,1.377032,1.845561,0.287979,0,0,1,0,0,1
889,1,1,26.000000,-0.680021,-0.560265,0.529354,1,0,0,1,0,0


In [12]:
y = data['Survived']
count_0 = sum(y == 0)  
count_1 = sum(y == 1) 
scale_pos_weight = count_0 / count_1

In [13]:
train=data.drop('Survived',axis=1)
test=data['Survived']
X_train, X_test, y_train, y_test = train_test_split(train, test, test_size=0.2, random_state=42)

In [16]:
rf = RandomForestClassifier(random_state=42)

param_dist = {
    'n_estimators': np.arange(50, 301, 50),
    'max_depth': [None] + list(np.arange(5, 31, 5)),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],  # no 'auto'
    'bootstrap': [True, False]
}

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,                 # Number of parameter settings sampled
    scoring='roc_auc',         # Optimize for ROC AUC
    cv=5,                     # 5-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1                 # Use all CPU cores
)

# Run hyperparameter search
random_search.fit(X_train, y_train)

# Best parameters
print("Best Hyperparameters:", random_search.best_params_)

# Evaluate on test set
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test)
y_proba = best_rf.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters: {'n_estimators': np.int64(300), 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': np.int64(20), 'bootstrap': True}
              precision    recall  f1-score   support

           0       0.83      0.91      0.87       105
           1       0.86      0.74      0.80        74

    accuracy                           0.84       179
   macro avg       0.85      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



In [17]:
best_params = {
    'n_estimators': int(np.int64(300)),
    'min_samples_split': 10,
    'min_samples_leaf': 1,
    'max_features': 'log2',
    'max_depth': int(np.int64(20)),
    'bootstrap': True,
    'random_state': 42  # Always fix random state for reproducibility
}

model = RandomForestClassifier(**best_params)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8435754189944135
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.91      0.87       105
           1       0.86      0.74      0.80        74

    accuracy                           0.84       179
   macro avg       0.85      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



In [26]:
test = pd.read_csv("test.csv")
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [27]:
test = test.drop(["Name","Ticket","Cabin"], axis=1)
test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,male,NaN,0,0,8.0500,S
414,1306,1,female,39.0,0,0,108.9000,C
415,1307,3,male,38.5,0,0,7.2500,S
416,1308,3,male,NaN,0,0,8.0500,S


In [28]:
test["Sex"] = test.Sex.map({"male": 1, "female": 0})
test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,Q
1,893,3,0,47.0,1,0,7.0000,S
2,894,2,1,62.0,0,0,9.6875,Q
3,895,3,1,27.0,0,0,8.6625,S
4,896,3,0,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,1,NaN,0,0,8.0500,S
414,1306,1,0,39.0,0,0,108.9000,C
415,1307,3,1,38.5,0,0,7.2500,S
416,1308,3,1,NaN,0,0,8.0500,S


In [29]:

# Reshape Age column to 2D
age_values = test[['Age']].values  # Ensures (n_samples, 1)

# Scale (improves imputation stability)
scaler_t = StandardScaler()
scaled_age = scaler_t.fit_transform(age_values)

# Iterative Imputation
imputer = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=0)
imputed_scaled_age = imputer.fit_transform(scaled_age)

# Inverse transform to return to original scale
imputed_age = scaler.inverse_transform(imputed_scaled_age)

# Replace missing Age values in original DataFrame
test['Age'] = imputed_age[:, 0]
test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.032956,0,0,7.8292,Q
1,893,3,0,46.847653,1,0,7.0000,S
2,894,2,1,62.225290,0,0,9.6875,Q
3,895,3,1,26.344137,0,0,8.6625,S
4,896,3,0,21.218258,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,1,29.699118,0,0,8.0500,S
414,1306,1,0,38.646247,0,0,108.9000,C
415,1307,3,1,38.133659,0,0,7.2500,S
416,1308,3,1,29.699118,0,0,8.0500,S


In [30]:
test, kurtosis_summary = reduce_kurtosis_without_scipy(test)
print(kurtosis_summary)
print(test)

Transforming column 'SibSp' with kurtosis 26.50
Transforming column 'Parch' with kurtosis 31.41
Transforming column 'Fare' with kurtosis 17.92
             Kurtosis Before  Kurtosis After
Parch              31.412513       -0.242813
SibSp              26.498712       -1.390028
Fare               17.921595       -1.288631
Age                 0.883176        0.883176
PassengerId        -1.200000       -1.200000
Pclass             -1.382666       -1.382666
Sex                -1.684332       -1.684332
     PassengerId  Pclass  Sex        Age     SibSp     Parch      Fare  \
0            892       3    1  34.032956 -0.689395 -0.538476 -1.066002   
1            893       3    0  46.847653  1.394759 -0.538476 -1.274818   
2            894       2    1  62.225290 -0.689395 -0.538476 -0.695892   
3            895       3    1  26.344137 -0.689395 -0.538476 -0.885825   
4            896       3    0  21.218258  1.394759  1.810664 -0.323766   
..           ...     ...  ...        ...       ...   

In [31]:
encoded = pd.get_dummies(test['Embarked'], prefix='Embarked').astype(int)
test = pd.concat([test.drop('Embarked', axis=1), encoded], axis=1)
test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,892,3,1,34.032956,-0.689395,-0.538476,-1.066002,0,1,0
1,893,3,0,46.847653,1.394759,-0.538476,-1.274818,0,0,1
2,894,2,1,62.225290,-0.689395,-0.538476,-0.695892,0,1,0
3,895,3,1,26.344137,-0.689395,-0.538476,-0.885825,0,0,1
4,896,3,0,21.218258,1.394759,1.810664,-0.323766,0,0,1
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,29.699118,-0.689395,-0.538476,-1.015657,0,0,1
414,1306,1,0,38.646247,-0.689395,-0.538476,1.564411,1,0,0
415,1307,3,1,38.133659,-0.689395,-0.538476,-1.208299,0,0,1
416,1308,3,1,29.699118,-0.689395,-0.538476,-1.015657,0,0,1


In [32]:
encoded = pd.get_dummies(test['Pclass'], prefix='Pclass').astype(int)
test = pd.concat([test.drop('Pclass', axis=1), encoded], axis=1)
test

,PassengerId,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,892,1,34.032956,-0.689395,-0.538476,-1.066002,0,1,0,0,0,1
1,893,0,46.847653,1.394759,-0.538476,-1.274818,0,0,1,0,0,1
2,894,1,62.225290,-0.689395,-0.538476,-0.695892,0,1,0,0,1,0
3,895,1,26.344137,-0.689395,-0.538476,-0.885825,0,0,1,0,0,1
4,896,0,21.218258,1.394759,1.810664,-0.323766,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,1,29.699118,-0.689395,-0.538476,-1.015657,0,0,1,0,0,1
414,1306,0,38.646247,-0.689395,-0.538476,1.564411,1,0,0,1,0,0
415,1307,1,38.133659,-0.689395,-0.538476,-1.208299,0,0,1,0,0,1
416,1308,1,29.699118,-0.689395,-0.538476,-1.015657,0,0,1,0,0,1


In [33]:
feature_columns = [col for col in test.columns if col !="PassengerId"]
X_test = test[feature_columns]
y_test_pred = model.predict(X_test) 
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"].astype(int),
    "Survived": y_test_pred
})
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
submission


TypeError: 'tuple' object is not callable

In [37]:
submission.to_csv("gender_submission.csv",index = False)